In [ ]:
import sys, os
project_root = os.getcwd()
sys.path.append(project_root[:-6])
import numpy as np
from region_division.obstacles.mountain_generator import generate_mountains
from region_division.obstacles.maze_generator import render_maze, generate_maze
from sector_allocation import constrained_geodesic_kmeans
from path_planning import build_trajectories_by_sectors
from visualization import (
    visualize_sectors_and_trajectories,
    visualize_single_agent_trajectory,
    visualize_layer_coverage,
    save_figure_as_png
)

def main():
    # Параметры карты
    width, height = 100, 100
    depth = 20
    
    print("Генерация карты местности...")
    
    # Генерация горной местности
    mountain_map = generate_mountains(5, (width, height, depth))
    
    # Генерация лабиринта
    removed_walls = generate_maze(10, 10, add_connections=5)
    maze_map = render_maze(removed_walls, 10, 10, 5, 2, 15)
    
    # Комбинация обоих типов препятствий
    combined_map = np.zeros_like(mountain_map)
    combined_map[:maze_map.shape[0], :maze_map.shape[1], :maze_map.shape[2]] = maze_map
    combined_map = np.logical_or(combined_map, mountain_map).astype(np.int8)
    
    # Используем комбинированную карту
    map_3d = combined_map
    
    # Определение начальных позиций агентов
    num_agents = 3
    
    # Находим свободные позиции для агентов
    agent_positions = []
    free_cells = np.where(map_3d == 0)
    # Выбираем индексы с большим разбросом
    free_indices = np.array(range(len(free_cells[0])))
    np.random.shuffle(free_indices)
    
    # Берем индексы с равным интервалом для лучшего распределения
    interval = len(free_indices) // num_agents
    indices = [free_indices[i * interval] for i in range(num_agents)]
    
    for i in range(num_agents):
        agent_positions.append([
            free_cells[2][indices[i]],  # X
            free_cells[1][indices[i]],  # Y
            free_cells[0][indices[i]]   # Z
        ])
    
    agent_positions = np.array(agent_positions)
    
    print(f"Выбраны начальные позиции для {num_agents} агентов")
    
    # Разделение пространства на сектора с использованием улучшенного алгоритма
    regions = constrained_geodesic_kmeans(map_3d, num_agents, agent_positions)
    
    # Построение траекторий для каждого агента в своем секторе
    trajectories = build_trajectories_by_sectors(map_3d, regions, agent_positions)
    
    # Визуализация результатов
    print("Создание визуализаций...")
    
    # Общая визуализация всех секторов и траекторий
    fig_all = visualize_sectors_and_trajectories(map_3d, regions, trajectories, agent_positions)
    fig_all.show()
    save_figure_as_png(fig_all, "все_сектора_и_траектории")
    
    # Визуализация траектории каждого агента отдельно
    for agent_id in range(num_agents):
        if trajectories[agent_id]:
            fig_agent = visualize_single_agent_trajectory(
                map_3d, regions, trajectories[agent_id], agent_positions[agent_id], agent_id)
            fig_agent.show()
            save_figure_as_png(fig_agent, f"агент_{agent_id}_траектория")
    
    # Визуализация покрытия отдельных слоев
    # Визуализируем несколько слоев для лучшего понимания
    for layer_idx in [depth // 4, depth // 2, 3 * depth // 4]:
        # Проверяем, есть ли траектории в этом слое
        has_coverage = False
        for trajectory in trajectories:
            if trajectory:
                traj_array = np.array(trajectory)
                layer_mask = np.abs(traj_array[:, 2] - layer_idx) < 0.5
                if np.any(layer_mask):
                    has_coverage = True
                    break
        
        if has_coverage:
            fig_layer = visualize_layer_coverage(map_3d, regions, trajectories, layer_idx)
            fig_layer.show()
            save_figure_as_png(fig_layer, f"покрытие_слоя_{layer_idx}")
        else:
            print(f"Нет покрытия в слое {layer_idx}, пропускаем визуализацию")
    
    print("Готово! Визуализации сохранены как PNG-файлы.")

if __name__ == "__main__":
    main()